<table style='border: none' align='left'>
   <tr style='border: none'>
      <th style='border: none'><font face='verdana' size='5' color='black'><b>Iris multi-class classification with scikit-learn</b></th>
      <th style='border: none'><img src='https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true' alt='Watson Machine Learning icon' height='40' width='40'></th>
   </tr>
   <tr style='border: none'>
       <th style='border: none'><img src='https://cloud.ibm.com/docs-content/v1/content/6b4d680b3820820b97fbf9b7db3c9d3658db8ac2/tutorials/images/solution22-build-machine-learning-model/iris_machinelearning.png' width='600' alt='Icon'> </th>
   </tr>
</table>

**Note:** _The notebook is part of a solution tutorial [Build, deploy, test, retrain and monitor a predictive machine learning model](https://cloud.ibm.com/docs/tutorials?topic=solution-tutorials-create-deploy-retrain-machine-learning-model) and used for demonstrating IBM Watson OpenScale only._

This notebook utilizes <a href="https://pypi.python.org/pypi/watson-machine-learning-client" target="_blank" rel="noopener noreferrer">watson-machine-learning-client</a> package in order to save, deploy, and score a predictive model.

Some familiarity with Python is helpful. This notebook uses Python 3.6, scikit-learn, and the Watson Machine Learning (WML) API client (`watson-machine-learning-client`).

You will use the sample **iris flower data set**.The Iris dataset was used in R.A. Fisher's classic 1936 paper, [The Use of Multiple Measurements in Taxonomic Problems](http://rcs.chemometrics.ru/Tutorials/classification/Fisher.pdf), and can also be found on the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/). This small dataset is often used for testing out machine learning algorithms and visualizations. The aim is to classify Iris flowers among three species (Setosa, Versicolor or Virginica) from measurements of length and width of sepals and petals. The iris data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant.

<a id='setup'></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>)

- Configure your local python environment:
  + python 3.6
  + scikit-learn 0.20.0
  + watson-machine-learning-client

**Tip**: Run the cell below to install packages from <a href='https://pypi.python.org/pypi' target='_blank' rel='noopener no referrer'>PyPI</a>.

In [1]:
# Remove current wml package files.
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [2]:
# Install watson-machine-learning-client required for online deployment and scoring.
!pip install --user watson-machine-learning-client --upgrade

     |████████████████████████████████| 542kB 22.4MB/s eta 0:00:01


<a id='load'></a>
## Load and explore data

In this section, you will load the data from a sample iris `.csv` file

In [3]:
!pip install scikit-learn==0.20.0

     |████████████████████████████████| 5.3MB 9.8MB/s eta 0:00:01
ERROR: autoai-libs 1.10.5 has requirement numpy>=1.16.4, but you'll have numpy 1.15.4 which is incompatible.
ERROR: autoai-libs 1.10.5 has requirement pandas>=0.24.2, but you'll have pandas 0.24.1 which is incompatible.
ERROR: autoai-libs 1.10.5 has requirement scikit-learn==0.20.3, but you'll have scikit-learn 0.20.0 which is incompatible.
  Found existing installation: scikit-learn 0.20.3
    Uninstalling scikit-learn-0.20.3:
      Successfully uninstalled scikit-learn-0.20.3


In [4]:
import pandas as pd

iris = pd.read_csv("https://raw.githubusercontent.com/IBM-Cloud/ml-iris-classification/master/data/iris.csv")
# download the file
print(iris.shape)
print(iris.columns)

(150, 5)
Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')


The sample data set consists of 8x8 pixel images of hand-written digits.

In [5]:
iris.shape

(150, 5)

Display the first digit data and label using **data** and **target**. The hand-written digit shown below is an 8x8 pixel image represented by an array.

In [6]:
iris.describe

<bound method NDFrame.describe of      sepal_length  sepal_width  petal_length  petal_width    species
0             5.1          3.5           1.4          0.2     setosa
1             4.9          3.0           1.4          0.2     setosa
2             4.7          3.2           1.3          0.2     setosa
3             4.6          3.1           1.5          0.2     setosa
4             5.0          3.6           1.4          0.2     setosa
5             5.4          3.9           1.7          0.4     setosa
6             4.6          3.4           1.4          0.3     setosa
7             5.0          3.4           1.5          0.2     setosa
8             4.4          2.9           1.4          0.2     setosa
9             4.9          3.1           1.5          0.1     setosa
10            5.4          3.7           1.5          0.2     setosa
11            4.8          3.4           1.6          0.2     setosa
12            4.8          3.0           1.4          0.1     setosa


<a id='model'></a>
## Create a scikit-learn model

### Split data<a id='prep'></a>

In this section you will split your data into: 
- Train data set
- Test data set

In [7]:
from sklearn.model_selection import train_test_split
iris_training, iris_test = train_test_split(iris, test_size=0.2)
# separate x and y variables from data
iris_training_y = iris_training["species"]
iris_training_x = iris_training.drop("species", axis=1)

iris_test_y = iris_test["species"]
iris_test_x = iris_test.drop("species", axis=1)

In [8]:
print(iris_training_x.count())

sepal_length    120
sepal_width     120
petal_length    120
petal_width     120
dtype: int64


### Train a model<a id='train'></a>

In [9]:
from sklearn import tree
classifier=tree.DecisionTreeClassifier()

In [10]:
classifier.fit(iris_training_x,iris_training_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### Test the model

In [11]:
predictions=classifier.predict(iris_test_x)

In [12]:
from sklearn.metrics import accuracy_score
print(accuracy_score(iris_test_y,predictions))

0.9333333333333333


**Note:** You can tune your model, or try creating classification models to achieve better accuracy. For simplicity of this example, the tuning section and model selection are omitted.

<a id='persistence'></a>
## Save, load, and delete a model in the WML repository

In this section, you will learn how to use the `watson-machine-learning-client` package to manage your model in the WML repository.

**Tip**: You can find more information about the watson-machine-learning-client <a href="https://console.bluemix.net/docs/services/service_credentials.html#service_credentials" target="_blank" rel="noopener no referrer">here</a>[](https://wml-api-pyclient.mybluemix.net).

### Set up the WML instance<a id='work'></a>

First, import required modules.

In [13]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**Note**: A deprecation warning is returned from scikit-learn package that does not impact watson machine learning client functionalities.
Authenticate to the Watson Machine Learning service on the IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-get-wml-credentials.html" target="_blank" rel="noopener no referrer">Service credentials</a> tab of the service instance that you created on the IBM Cloud. <BR>If you cannot find the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here without any trailing spaces.


In [14]:
wml_credentials = {
    "apikey": "<API_KEY>",
    "instance_id": "<INSTANCE_ID>",
    "url": "<URL>"
}

#### Instantiate the WML API client object. 

In [15]:
client = WatsonMachineLearningAPIClient(wml_credentials)

#### Get instance details.

In [16]:
import json

instance_details = client.service_instance.get_details()

### Save the model in the WML repository<a id='save'></a>

Define the model name when saving the model in the WML repository.

In [17]:
published_model = client.repository.store_model(model=classifier, meta_props={'name':'scikit iris model'}, \
                                                training_data=iris_training_x, training_target=iris_training_y)

#### Get information about a specific model in the WML repository.

In [18]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "guid": "86886f03-00ae-4de2-8200-958c45f00459",
    "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/e97f9e84-eefc-4eb8-bb6d-7dbfc33bb536/published_models/86886f03-00ae-4de2-8200-958c45f00459",
    "created_at": "2020-04-21T06:15:54.672Z",
    "modified_at": "2020-04-21T06:15:54.744Z"
  },
  "entity": {
    "runtime_environment": "python-3.6",
    "learning_configuration_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/e97f9e84-eefc-4eb8-bb6d-7dbfc33bb536/published_models/86886f03-00ae-4de2-8200-958c45f00459/learning_configuration",
    "name": "scikit iris model",
    "label_col": "species",
    "learning_iterations_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/e97f9e84-eefc-4eb8-bb6d-7dbfc33bb536/published_models/86886f03-00ae-4de2-8200-958c45f00459/learning_iterations",
    "training_data_schema": {
      "features": {
        "type": "DataFrame",
        "fields": [
          {
            "name": "sepal_length",
            "typ

#### Get information of all the models in the WML repository.

In [19]:
models_details = client.repository.list_models()

------------------------------------  -----------------  ------------------------  -----------------
GUID                                  NAME               CREATED                   FRAMEWORK
86886f03-00ae-4de2-8200-958c45f00459  scikit iris model  2020-04-21T06:15:54.672Z  scikit-learn-0.20
------------------------------------  -----------------  ------------------------  -----------------


### Load a model from the WML repository<a id='load_model'></a>

In this subsection, you will learn how to load a saved model from a specific WML instance.

In [20]:
loaded_model = client.repository.load(published_model_uid)

Make test predictions to check that the model has been loaded correctly.

In [21]:
test_predictions = loaded_model.predict(iris_test_x[:10])
print(iris_test_x[:10])

     sepal_length  sepal_width  petal_length  petal_width
82            5.8          2.7           3.9          1.2
134           6.1          2.6           5.6          1.4
114           5.8          2.8           5.1          2.4
42            4.4          3.2           1.3          0.2
109           7.2          3.6           6.1          2.5
57            4.9          2.4           3.3          1.0
1             4.9          3.0           1.4          0.2
70            5.9          3.2           4.8          1.8
25            5.0          3.0           1.6          0.2
84            5.4          3.0           4.5          1.5


In [22]:
# The predictions made by the model.
print(test_predictions)

['versicolor' 'virginica' 'virginica' 'setosa' 'virginica' 'virginica'
 'setosa' 'virginica' 'setosa' 'versicolor']


As you can see, you are able to make predictions which means that the model has loaded correctly. You have now learned how save and load the model in/from the WML repository.

### Delete a model from the WML repository<a id='delete'></a>

The code in the following cell deletes a published model from the WML repository. The code is not executable at this stage because you still need the model for deployment.

<a id='scoring'></a>
## Deploy and score data in the WML repository

In this section, you will learn how to use the WML API client to perform online scoring and score a test data record.

### Create an online deployment for the published model<a id='create'></a>

In [23]:
created_deployment = client.deployments.create(published_model_uid, 'Deployment of scikit model')



#######################################################################################

Synchronous deployment creation for uid: '86886f03-00ae-4de2-8200-958c45f00459' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='916af631-f34f-4e29-951d-baf5bd2eb964'
------------------------------------------------------------------------------------------------




**Note**: You use the deployment url saved in published_model_uid object. The next section shows you how to retrieve the deployment url from the WML instance.

Now, you can define and print an online scoring endpoint. 

 5.2 Get deployments<a id='getdeploy'></a>

In [24]:
deployments = client.deployments.get_details()
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/e97f9e84-eefc-4eb8-bb6d-7dbfc33bb536/deployments/916af631-f34f-4e29-951d-baf5bd2eb964/online


You can get the deployment_url by parsing the deployment details for the last deployed model.

### Score data<a id='score'></a>

Use the following method to run a test scoring request against the deployed model.

**Action**: Prepare scoring payload with the records to score.

In [2]:
import random
for i in range(100):
    array_of_values_to_be_scored = [round(random.uniform(0.0, 10.0), 1), round(random.uniform(
        0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1)]
    another_array_of_values_to_be_scored = [round(random.uniform(0.0, 10.0), 1), round(random.uniform(
        0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1), round(random.uniform(0.0, 10.0), 1)]
    scoring_data = {
        'fields': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], 
        'values': [array_of_values_to_be_scored,another_array_of_values_to_be_scored]
    }
    predictions = client.deployments.score(scoring_endpoint, scoring_data)

NameError: name 'client' is not defined

Use the ``client.deployments.score()`` method to run the scoring.

###  Delete the deployment<a id='deldeploy'></a>

Use the following method to delete the deployment.

You can check that your deployment was deleted by generating a list of your saved deployments:

### Delete the model<a id='delmodel'></a>

You can check that your model was deleted by generating a list of your saved models:

<a id='summary'></a>
## Summary and next steps     

<div style='background:#F5F7FA; height:110px; padding: 2em; font-size:14px;'>
<span style='font-size:18px;color:#152935;'>Love this notebook? </span>
<span style='font-size:15px;color:#152935;float:right;margin-right:40px;'>Don't have an account yet?</span><br>
<span style='color:#5A6872;'>Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style='border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;'><a href='https://ibm.co/wsnotebooks' target='_blank' style='color: #3d70b2;text-decoration: none;'>Sign Up</a></span><br>
</div>